In [1]:
import bct
import copy
import matplotlib.cm as cmx
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import utils

import warnings; warnings.simplefilter('ignore')

from atlases import DesikanAtlas
from hmmlearn import hmm
from sklearn.decomposition import PCA
from sklearn.externals import joblib

# Low Dimensional Connectome Dynamics

### Prepare Data - Separate Modalities

In [2]:
all_subjects_all_trials_connectomes = utils.load_connectomes(utils.ALL_SUBJECT_IDS, utils.ALL_TRIAL_IDS)
all_subjects_all_trials_connectomes['fmri'].shape

(17906, 68, 68)

Extract flattened representation of upper triangular of Pearson correlation matrix for each connectome type.

In [3]:
# NOTE: The below logic would have to change if we move away from using Desikan Atlas where the number of regions 
# are the same between EEG and fMRI
num_regions = all_subjects_all_trials_connectomes['fmri'].shape[1]
num_regions

68

In [5]:
upper_triangular_including_diagonal_idxs = np.triu_indices(num_regions, k=0)
lower_triangular_idxs = np.tril_indices(num_regions, k=-1)

In [6]:
all_subjects_all_trials_connectome_upper_triangular_flattened = copy.deepcopy(all_subjects_all_trials_connectomes)
for k in all_subjects_all_trials_connectome_upper_triangular_flattened:
    all_subjects_all_trials_connectome_upper_triangular_flattened[k] = np.array([c[upper_triangular_including_diagonal_idxs].flatten() for c in all_subjects_all_trials_connectomes[k]])

In [7]:
all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'].shape

(17906, 2346)

### Prepare Data - Combined Modalities

In [8]:
def data_matrix_from_channels(channels):
    data_matrix = []
    for k in channels:
        data_matrix.append(all_subjects_all_trials_connectome_upper_triangular_flattened[k])
    data_matrix = np.concatenate(data_matrix, axis=1)
    return data_matrix

In [9]:
alpha_beta_delta_gamma_theta_matrix = data_matrix_from_channels(['alpha', 'beta', 'delta', 'gamma', 'theta'])
fmri_alpha_beta_delta_gamma_theta_matrix = data_matrix_from_channels(['fmri', 'alpha', 'beta', 'delta', 'gamma', 'theta'])
fmri_broad_matrix = data_matrix_from_channels(['fmri', 'broad'])

In [ ]:
# num_features_per_modality = all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'].shape[1]
# fmri_feature_idxs  = range(num_features_per_modality*0, num_features_per_modality*1)
# alpha_feature_idxs = range(num_features_per_modality*1, num_features_per_modality*2)
# beta_feature_idxs  = range(num_features_per_modality*2, num_features_per_modality*3)
# delta_feature_idxs = range(num_features_per_modality*3, num_features_per_modality*4)
# gamma_feature_idxs = range(num_features_per_modality*4, num_features_per_modality*5)
# theta_feature_idxs = range(num_features_per_modality*5, num_features_per_modality*6)

# feature_idxs = [('fmri', fmri_feature_idxs),
#                 ('alpha', alpha_feature_idxs),
#                 ('beta', beta_feature_idxs),
#                 ('delta', delta_feature_idxs),
#                 ('gamma', gamma_feature_idxs),
#                 ('theta', theta_feature_idxs)]



## Hidden Markov Models

### Training

(1) TODO: Find a bunch of graph statistics (node-wise or global) and compute the time series of the graph statistics and fit an HMM to that.

(2) TODO: Train HMMs on subset of data and predict likelihood of new data

(3) TODO: Align states between modalities.

In [13]:
def num_parameters_in_hmm(model):  
    k = model.means_.shape[0]
    d = model.means_.shape[1]
    # NOTE: We are using a diagonal covariance matrix
    return (k*d)+(k*d)+k+(k**2)

In [14]:
def train_optimal_hmm_on_data(data, bic=True, pca_variance_retained=0.99, forced_component_count=None):
    
    # Apply PCA to dimensionality reduce the data and retain xx% variance
    print("\tApplying pca and retaining {0:.2f}% variance...".format(pca_variance_retained*100))
    pca_model = PCA(pca_variance_retained)
    dim_reduced_data = pca_model.fit_transform(data)
    print("\t{0} dimensions -> {1} dimensions".format(data.shape[1], dim_reduced_data.shape[1]))
    
    # Compute BIC/AIC scores for HMM models with # states between 1 and N-1
    scores = []
    candidate_hmm_models = []
    
    for n_components in (range(2, data.shape[0]-1) if not forced_component_count else range(forced_component_count, forced_component_count+1)):
        
        print("\t\t training hmm with n_components={0}".format(n_components))

        # Train HMM model
        candidate_hmm_model = hmm.GaussianHMM(n_components=n_components,
                                              covariance_type="diag").fit(dim_reduced_data)
        candidate_hmm_models.append(candidate_hmm_model)

        # Compute BIC/AIC score
        n = dim_reduced_data.shape[0]
        k = num_parameters_in_hmm(candidate_hmm_model)
        L = candidate_hmm_model.decode(dim_reduced_data)[0]

        print("\t\t\tn={0}, k={1}, L={2}".format(n, k, L))

        score = (np.log(n)*k - 2*L) if bic else (k - L)
        scores.append(score)
        print("\t\t\tScore={0}".format(score))
        
        if len(scores) >= 3:
            # Last two increases in number of components yielded worse scores -> exit search
            if scores[-1] > scores[-3] and scores[-2] > scores[-3]:
                break
    
    # Select the model with the lowest BIC/AIC score
    selected_hmm_model = candidate_hmm_models[np.argmin(scores)]
    return selected_hmm_model, pca_model

In [15]:
def train_optimal_hmms_together(data_channels, bic=True, pca_variance_retained=0.99):
    
    # Apply PCA to dimensionality reduce the data and retain xx% variance
    print("\tApplying pca and retaining {0:.2f}% variance...".format(pca_variance_retained*100))
    pca_models = []
    dim_reduced_data_channels = []
    
    for data in data_channels:
        pca_model = PCA(pca_variance_retained)
        dim_reduced_data = pca_model.fit_transform(data)
        
        pca_models.append(pca_model)
        dim_reduced_data_channels.append(dim_reduced_data)
        print("\t{0} dimensions -> {1} dimensions".format(data.shape[1], dim_reduced_data.shape[1]))
    
    # Compute BIC/AIC scores for HMM models with # states between 1 and N-1
    scores = []
    candidate_hmm_models = []
    
    for n_components in range(2, len(data_channels[0])-1):
        
        print("\t\t training hmms with n_components={0}".format(n_components))

        # Train HMM model for every data channel
        candidate_hmm_models_with_n_components = []
        scores_with_n_components = []
        for dim_reduced_data in dim_reduced_data_channels:
            candidate_hmm_model = hmm.GaussianHMM(n_components=n_components,
                                                  covariance_type="diag").fit(dim_reduced_data)
            candidate_hmm_models_with_n_components.append(candidate_hmm_model)
            
            # Compute BIC/AIC score
            n = dim_reduced_data.shape[0]
            k = num_parameters_in_hmm(candidate_hmm_model)
            L = candidate_hmm_model.decode(dim_reduced_data)[0]

            print("\t\t\t\tn={0}, k={1}, L={2}".format(n, k, L))

            score = (np.log(n)*k - 2*L) if bic else (k - L)
            scores_with_n_components.append(score)
            print("\t\t\t\t\t=> Score={0}".format(score))
        
        candidate_hmm_models.append(candidate_hmm_models_with_n_components)
        scores.append(np.mean(scores_with_n_components))
        print("\t\t\tScore={0}".format(scores[-1]))


        if len(scores) >= 3:
            # Last two increases in number of components yielded worse scores -> exit search
            if scores[-1] > scores[-3] and scores[-2] > scores[-3]:
                break
                
    # Select the model with the lowest BIC/AIC score
    selected_hmm_models = candidate_hmm_models[np.argmin(scores)]
    return selected_hmm_models, pca_models

##### COMBINED EEG BANDS (Alpha+Beta+Delta+Gamma+Theta)

In [20]:
combined_alpha_beta_delta_gamma_theta_hmm_model = joblib.load("output/hmm/models/[hmm]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))
combined_alpha_beta_delta_gamma_theta_pca_model = joblib.load("output/hmm/models/[pca]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))

##### COMBINED EEG BANDS (Broad)

In [28]:
broad_hmm_model = joblib.load("output/hmm/models/[hmm]-[broad].pkl")
broad_pca_model = joblib.load("output/hmm/models/[pca]-[broad].pkl")

#####  COMBINED MODALITY HMM MODEL (fMRI+Alpha+Beta+Delta+Gamma+Theta)

In [17]:
combined_fmri_alpha_beta_delta_gamma_theta_hmm_model = joblib.load("output/hmm/models/[hmm]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))
combined_fmri_alpha_beta_delta_gamma_theta_pca_model = joblib.load("output/hmm/models/[pca]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))

#####  COMBINED MODALITY HMM MODEL (fMRI+Broad)

In [ ]:
combined_fmri_broad_hmm_model = joblib.load("output/hmm/models/[hmm]-[fmri_broad]-[{0}].pkl".format("combined"))
combined_fmri_broad_pca_model = joblib.load("output/hmm/models/[pca]-[fmri_broad]-[{0}].pkl".format("combined"))

##### SEPARATE EEG BANDS (Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [19]:
alpha_beta_delta_gamma_theta_hmm_models = [joblib.load("output/hmm/models/[hmm]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)) for k in ['alpha', 'beta', 'delta', 'gamma', 'theta']]
alpha_beta_delta_gamma_theta_pca_models = [joblib.load("output/hmm/models/[pca]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)) for k in ['alpha', 'beta', 'delta', 'gamma', 'theta']]

##### SEPARATE MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [16]:
fmri_alpha_beta_delta_gamma_theta_pca_models = []
fmri_alpha_beta_delta_gamma_theta_hmm_models = []

for k in ['fmri', 'alpha', 'beta', 'delta', 'gamma', 'theta']:
    fmri_alpha_beta_delta_gamma_theta_pca_models.append(joblib.load("output/hmm/models/[pca]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)))
    fmri_alpha_beta_delta_gamma_theta_hmm_models.append(joblib.load("output/hmm/models/[hmm]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)))

##### SEPARATE MODALITIES (fMRI+Broad HMMs Trained Together)

In [44]:
fmri_broad_pca_models = []
fmri_broad_hmm_models = []

for k in ['fmri', 'broad']:
    fmri_broad_pca_models.append(joblib.load("output/hmm/models/[pca]-[fmri_broad]-[{0}].pkl".format(k)))
    fmri_broad_hmm_models.append(joblib.load("output/hmm/models/[hmm]-[fmri_broad]-[{0}].pkl".format(k)))

### Decoded Hidden State Sequence Likelihood Comparison

##### COMBINED EEG BANDS (Alpha+Beta+Delta+Gamma+Theta)

In [24]:
combined_alpha_beta_delta_gamma_theta_hmm_model.means_.shape[0]

4

In [21]:
combined_alpha_beta_delta_gamma_theta_log_likelihood, combined_alpha_beta_delta_gamma_theta_decoded_state_sequence = combined_alpha_beta_delta_gamma_theta_hmm_model.decode(combined_alpha_beta_delta_gamma_theta_pca_model.transform(alpha_beta_delta_gamma_theta_matrix))
combined_alpha_beta_delta_gamma_theta_log_likelihood

82461646.03284961

##### COMBINED EEG BANDS (Broad)

In [29]:
broad_hmm_model.means_.shape[0]

5

In [30]:
broad_log_likelihood, broad_decoded_state_sequence = broad_hmm_model.decode(broad_pca_model.transform(all_subjects_all_trials_connectome_upper_triangular_flattened['broad']))
broad_log_likelihood

40006049.92391768

##### COMBINED MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta)

In [25]:
combined_fmri_alpha_beta_delta_gamma_theta_hmm_model.means_.shape[0]

5

In [22]:
combined_fmri_alpha_beta_delta_gamma_theta_log_likelihood, combined_fmri_alpha_beta_delta_gamma_theta_decoded_state_sequence = combined_fmri_alpha_beta_delta_gamma_theta_hmm_model.decode(combined_fmri_alpha_beta_delta_gamma_theta_pca_model.transform(fmri_alpha_beta_delta_gamma_theta_matrix))
combined_fmri_alpha_beta_delta_gamma_theta_log_likelihood

2309626.5700253514

#####  COMBINED MODALITIES (fMRI+Broad)

In [33]:
combined_fmri_broad_hmm_model.means_.shape[0]

65

In [34]:
combined_fmri_broad_log_likelihood, combined_fmri_broad_decoded_state_sequence = combined_fmri_broad_hmm_model.decode(combined_fmri_broad_pca_model.transform(fmri_broad_matrix))
combined_fmri_broad_log_likelihood

-6776413.256126671

##### SEPARATE EEG BANDS (Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [38]:
alpha_beta_delta_gamma_theta_hmm_models[0].means_.shape[0]

6

In [36]:
alpha_beta_delta_gamma_theta_decoded_state_sequences = []
alpha_beta_delta_gamma_theta_decoded_log_likelihoods = []
alpha_beta_delta_gamma_theta_data_channels = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['alpha'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['beta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['delta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['gamma'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['theta']
]
for (data_channel, hmm_model, pca_model) in zip(alpha_beta_delta_gamma_theta_data_channels, alpha_beta_delta_gamma_theta_hmm_models, alpha_beta_delta_gamma_theta_pca_models):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    alpha_beta_delta_gamma_theta_decoded_log_likelihoods.append(decoded_state_sequence[0])
    alpha_beta_delta_gamma_theta_decoded_state_sequences.append(decoded_state_sequence[1])

In [37]:
print(alpha_beta_delta_gamma_theta_decoded_log_likelihoods)
print(np.mean(alpha_beta_delta_gamma_theta_decoded_log_likelihoods))

[25168602.111413915, 37566808.92075627, 26784010.713776994, 28916582.493297048, 25957636.951635607]
28878728.238175966


##### SEPARATE MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [45]:
fmri_alpha_beta_delta_gamma_theta_hmm_models[0].means_.shape[0]

6

In [39]:
fmri_alpha_beta_delta_gamma_theta_decoded_state_sequences = []
fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods = []
fmri_alpha_beta_delta_gamma_theta_data_channels = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['alpha'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['beta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['delta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['gamma'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['theta']
]
for (data_channel, hmm_model, pca_model) in zip(fmri_alpha_beta_delta_gamma_theta_data_channels, fmri_alpha_beta_delta_gamma_theta_hmm_models, fmri_alpha_beta_delta_gamma_theta_pca_models):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods.append(decoded_state_sequence[0])
    fmri_alpha_beta_delta_gamma_theta_decoded_state_sequences.append(decoded_state_sequence[1])

In [40]:
print(fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods)
print(np.mean(fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods))

[-7404563.869950348, 25168736.860782143, 37566920.50689304, 26777941.281714328, 28916600.5088779, 25957892.799562633]
22830588.014646616


##### SEPARATE MODALITIES (fMRI+Broad HMMs Trained Together)

In [46]:
fmri_broad_hmm_models[0].means_.shape[0]

24

In [50]:
fmri_broad_decoded_state_sequences = []
fmri_broad_decoded_log_likelihoods = []
fmri_broad_data_channels = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['broad'],
]
for (data_channel, hmm_model, pca_model) in zip(fmri_broad_data_channels, fmri_broad_hmm_models, fmri_broad_pca_models):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    fmri_broad_decoded_log_likelihoods.append(decoded_state_sequence[0])
    fmri_broad_decoded_state_sequences.append(decoded_state_sequence[1])

In [51]:
print(fmri_broad_decoded_log_likelihoods)
print(np.mean(fmri_broad_decoded_log_likelihoods))

[-7172960.976526341, 40144751.55825858]
16485895.290866118


### Spatial Representation of Hidden States

SEPARATE MODALTIES - Plot spatial representation of each hidden markov model state in connectome space.

In [ ]:
fig = plt.figure(figsize=(180, 60))
fig.suptitle('Spatial Loadings of Hidden Markov Model States', fontsize=40)

num_states_to_plot = 5
subplot_idx = 1
for (k, hmm_model, pca_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models, hmm_preprocessing_pca_models):
    for state_idx in range(0, num_states_to_plot):
        
        if hmm_model.means_.shape[0] <= state_idx:
            ax = fig.add_subplot(len(hmm_models), num_states_to_plot, subplot_idx)
            subplot_idx += 1
            continue

        # Extract connectome representation of the hidden state
        hidden_state = np.zeros((num_regions, num_regions))
        hidden_state[upper_triangular_including_diagonal_idxs] = np.matmul(hmm_model.means_[state_idx], pca_model.components_)
        hidden_state[lower_triangular_idxs] = hidden_state.T[lower_triangular_idxs]

        # Plot connectome representation of the hidden state
        ax = fig.add_subplot(len(hmm_models), num_states_to_plot, subplot_idx)
        DesikanAtlas.plot(connectome=hidden_state,
                          title='{0} HiddenState-{1} Connectome'.format(k, state_idx+1),
                          axes=ax)
        subplot_idx += 1

plt.savefig('output/hmm/spatial_loadings.png')
plt.close()

COMBINED MODALITIES - Plot spatial representation of each hidden markov model state in connectome space.

In [ ]:
fig = plt.figure(figsize=(180, 60))
fig.suptitle('Spatial Loadings of Combined Hidden Markov Model States', fontsize=40)

num_states_to_plot = min(5, combined_modality_hmm_model.means_.shape[0])
subplot_idx = 1
for feature_desc, feature_idx in feature_idxs:
    for state_idx in range(num_states_to_plot):
        
        # Extract connectome representation of the hidden state
        hidden_state = np.zeros((num_regions, num_regions))
        hidden_state[upper_triangular_including_diagonal_idxs] = np.matmul(combined_modality_hmm_model.means_[state_idx], combined_modality_preprocessing_pca_model.components_)[feature_idx]
        hidden_state[lower_triangular_idxs] = hidden_state.T[lower_triangular_idxs]

        # Plot connectome representation of the hidden state
        ax = fig.add_subplot(len(feature_idxs), num_states_to_plot, subplot_idx)
        DesikanAtlas.plot(connectome=hidden_state,
                          title='{0} HiddenState-{1} Connectome'.format(feature_desc, state_idx+1),
                          axes=ax)
        subplot_idx += 1

plt.savefig('output/hmm/combined_modality_spatial_loadings.png')
plt.close()

### Decoded Hidden State Sequence Statistics

Plot fractional occupancy - the fraction of time spent in each state relative to the total duration

In [ ]:
# SEPARATE
fig = plt.figure(figsize=(20, 4))
fig.suptitle('Fractional Occupancy')
num_plots = len(decoded_state_sequences)
subplot_idx = 1

for (k, hmm_model, state_seq) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models, decoded_state_sequences):
    
    # Compute fractional occupancy
    fractional_occupancies_per_state = []
    for state_idx in range(0, hmm_model.means_.shape[0]):
        fractional_occupancy = len(state_seq[state_seq == state_idx])/len(state_seq)
        fractional_occupancies_per_state.append(fractional_occupancy)
        
    # Plot fractional occupancy per state
    fig.add_subplot(1, num_plots, subplot_idx)
    plt.title(k)
    x = np.arange(hmm_model.means_.shape[0])
    plt.bar(x, height=fractional_occupancies_per_state)
    plt.xticks(x, [str(x_i) for x_i in x])
    subplot_idx += 1
    
plt.savefig('output/hmm/fractional_occupancy.png')
plt.close()

In [ ]:
# COMBINED
# Compute fractional occupancy
fractional_occupancies_per_state = []
for state_idx in range(0, combined_modality_hmm_model.means_.shape[0]):
    fractional_occupancy = len(combined_modality_decoded_state_sequence[combined_modality_decoded_state_sequence == state_idx])/len(combined_modality_decoded_state_sequence)
    fractional_occupancies_per_state.append(fractional_occupancy)

# Plot fractional occupancy per state
plt.title('Combined Fractional Occupancy')
x = np.arange(combined_modality_hmm_model.means_.shape[0])
plt.bar(x, height=fractional_occupancies_per_state)
plt.xticks(x, [str(x_i) for x_i in x])
plt.savefig('output/hmm/combined_fractional_occupancy.png')
plt.close()

Plot mean life time - the time spent in a state before transitioning to a new state on average

In [ ]:
# SEPARATE
fig = plt.figure(figsize=(20, 4))
fig.suptitle('Mean Life Time')
num_plots = len(decoded_state_sequences)
subplot_idx = 1

for (k, hmm_model, state_seq) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models, decoded_state_sequences):

    # Compute mean life time per state
    mean_life_time_per_state = []
    for state_id in range(0, hmm_model.means_.shape[0]):
        
        # Count number of transitions out of state with state_id
        num_transitions_out_of_state_with_state_id = 0
        for i in range(0, len(state_seq)-1):
            if state_seq[i] == state_id and state_seq[i+1] != state_id:
                num_transitions_out_of_state_with_state_id += 1
        
        # Count total number of time points spent in state with state id
        num_time_points_in_state_with_state_id = len(state_seq[state_seq == state_id])
        
        # Compute mean life time
        mean_life_time = num_time_points_in_state_with_state_id/num_transitions_out_of_state_with_state_id
        mean_life_time_per_state.append(mean_life_time)
    
    
    # Plot mean life time per state
    fig.add_subplot(1, num_plots, subplot_idx)
    plt.title(k)
    x = np.arange(hmm_model.means_.shape[0])
    plt.bar(x, height=mean_life_time_per_state)
    plt.xticks(x, [str(x_i) for x_i in x])
    subplot_idx += 1
    
plt.savefig('output/hmm/mean_life_time.png')
plt.close()

In [ ]:
# COMBINED
# Compute mean life time per state
mean_life_time_per_state = []
for state_id in range(0, combined_modality_hmm_model.means_.shape[0]):

    # Count number of transitions out of state with state_id
    num_transitions_out_of_state_with_state_id = 0
    for i in range(0, len(combined_modality_decoded_state_sequence)-1):
        if combined_modality_decoded_state_sequence[i] == state_id and combined_modality_decoded_state_sequence[i+1] != state_id:
            num_transitions_out_of_state_with_state_id += 1

    # Count total number of time points spent in state with state id
    num_time_points_in_state_with_state_id = len(combined_modality_decoded_state_sequence[combined_modality_decoded_state_sequence == state_id])

    # Compute mean life time
    mean_life_time = num_time_points_in_state_with_state_id/num_transitions_out_of_state_with_state_id
    mean_life_time_per_state.append(mean_life_time)


# Plot mean life time per state
plt.title('Combined Mean Life Time')
x = np.arange(combined_modality_hmm_model.means_.shape[0])
plt.bar(x, height=mean_life_time_per_state)
plt.xticks(x, [str(x_i) for x_i in x])

plt.savefig('output/hmm/combined_mean_life_time.png')
plt.close()

Plot transition probabilities.

In [ ]:
# SEPARATE - Transition Probabilities
fig = plt.figure(figsize=(20, 4))
fig.suptitle('Transition Probabilities')
num_plots = len(hmm_models)
subplot_idx = 1

for (k, hmm_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models):
    
    fig.add_subplot(1, num_plots, subplot_idx)
    plt.imshow(hmm_model.transmat_, cmap='gist_heat')
    plt.title(k)
    plt.colorbar()
    subplot_idx += 1

plt.savefig('output/hmm/transition_probabilities.png')
plt.close()

In [ ]:
# COMBINED - Transition Probabilities    
plt.imshow(combined_modality_hmm_model.transmat_, cmap='gist_heat')
plt.title('Combined Transition Probabilities')
plt.colorbar()
plt.savefig('output/hmm/combined_transition_probabilities.png')
plt.close()

Plot transitions between decoded states as a time series.

In [ ]:
def get_n_colors(n):
    return [ cmx.rainbow(float(i)/n) for i in range(n) ]

In [ ]:
# Separate
fig = plt.figure(figsize=(300, 25))
fig.suptitle('Decoded Hidden State Sequences')

num_plots = len(hmm_models)
subplot_idx = 1

for (k, hmm_model, state_seq) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, hmm_models, decoded_state_sequences):
    
    print(k)
    fig.add_subplot(num_plots, 1, subplot_idx)
    component_colors = get_n_colors(hmm_model.means_.shape[0])
    x = 0
    for state in state_seq[:6000]:
        plt.axvline(x=x, color=component_colors[state])
        x += 1
        
    plt.title("{0}".format(k))
    plt.yticks([])
    subplot_idx += 1    

plt.subplots_adjust(hspace=0.5)
plt.savefig('output/hmm/decoded_hidden_state_sequence.png')
plt.close()

In [ ]:
# Combined
fig = plt.figure(figsize=(300, 5))

component_colors = get_n_colors(combined_modality_hmm_model.means_.shape[0])
x = 0
for state in combined_modality_decoded_state_sequence[:6000]:
    plt.axvline(x=x, color=component_colors[state])
    x += 1

plt.title("{0}".format("Combined Decoded Hidden State Sequence"))
plt.yticks([])
plt.subplots_adjust(hspace=0.5)
plt.savefig('output/hmm/combined_decoded_hidden_state_sequence.png')
plt.close()

# Brain Graph Statistics

In [ ]:
def normalize(data):
    max_v = max(data)
    min_v = min(data)
    return (np.array(data) - min_v) / (max_v - min_v)

In [ ]:
t = np.arange(len(all_subjects_all_trials_connectomes['fmri']))

# The assortativity coefficient is a correlation coefficient between the strengths (weighted degrees) of all nodes
# on two opposite ends of a link. A positive assortativity coefficient indicates that nodes tend to link to other
# nodes with the same or similar strength.
assortativity = []

global_efficiency = []

for time_pt in all_subjects_all_trials_connectomes['fmri'][:100]:
    assortativity.append(bct.assortativity_wei(time_pt))  
    global_efficiency.append(bct.efficiency_wei(time_pt))

In [ ]:
plt.plot(t[:100], normalize(assortativity), color="red", label="Assortativity")
plt.plot(t[:100], normalize(global_efficiency), color="blue", label="Global Efficiency")
plt.legend()
plt.savefig('output/hmm/graph_statistics_fmri.png')
plt.close()

# Visualize fMRI/EEG Connectome Dynamics

In [ ]:
# for subject_id in ALL_SUBJECT_IDS:
#     for trial_id in ALL_TRIAL_IDS:
        
#         # Attempt to load all connectome types
#         connectomes = load_all_connectome_types(subject_id, trial_id,
#                                                atlas='desikan', 
#                                                seconds_used_to_compute_fmri_connectome=60,
#                                                exclude_bad_fmri_frames=True,
#                                                filter_artifact_timepoints=True)
        
#         if connectomes is None:
#             continue

#         # Plot connectomes through time
#         for t in range(0, connectomes['fmri'].shape[0]):
            
#             # Create figure and set title
#             fig = plt.figure(figsize=(30, 35))
#             fig.suptitle('Subject: "{0}" | Trial: {1} | Time: {2}'.format(subject_id, trial_id, t), fontsize=50)
            
#             # Plot connectomes
#             subplot_idx = 1
#             for connectome_id, connectome in connectomes.items():

#                 ax = fig.add_subplot(len(connectomes), 2, subplot_idx)
#                 plotting.plot_connectome(connectome[t], desikan_atlas_coordinates(), title='{0} Connectome'.format(connectome_id),
#                                          edge_threshold='95%', node_size=20, colorbar=True, axes=ax)
#                 subplot_idx += 1
            
#                 ax = fig.add_subplot(len(connectomes), 2, subplot_idx)
#                 plotting.plot_matrix(connectome[t], vmin=-1., vmax=1., colorbar=True, axes=ax)
#                 subplot_idx += 1
    
#             plt.savefig('output/connectomes_through_time/subject={0}_trial={1}_t={2}.png'.format(subject_id, trial_id, t))